<a href="https://colab.research.google.com/github/asalroudbari/RoboticGrasping/blob/main/YOLOV8Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check software and hardware.

In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.166 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
from ultralytics import YOLO

from IPython.display import display, Image

In [2]:
#@title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.7/559.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 40.4 MB/s eta 0:00:00
Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


##Dataset

In [4]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/content/datasets


In [5]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="KdyzhxZx8EoSBwkDqpaO")
project = rf.workspace("taarlab-e2cvo").project("utensilgrasp")
dataset = project.version(3).download("yolov8")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.2 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=9f80b0aff301c4c18423350438b12c791f7bd68f0945f9f9f44021e2eff3e263
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.166, to fix: `pip install ultralytics==8.0.134`


Extracting Dataset Version Zip to UtensilGrasp-3 in yolov8:: 100%|██████████| 1528/1528 [00:00<00:00, 2751.31it/s]


# Freeze Layers

In [12]:
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze
    for k, v in model.named_parameters():
        v.requires_grad = True  # train all layers
        if any(x in k for x in freeze):
            print(f'freezing {k}')
            v.requires_grad = False
    print(f"{num_freeze} layers are freezed.")

In [13]:
%cd /content

/content


In [14]:
%cat {dataset.location}/data.yaml

names:
- bottle
- layed bottle
- metal box
- metal fork
- metal spoon
- pfork
- pglass
- pknife
- pspoon
- sause
- square plate
nc: 11
roboflow:
  license: CC BY 4.0
  project: utensilgrasp
  url: https://universe.roboflow.com/taarlab-e2cvo/utensilgrasp/dataset/3
  version: 3
  workspace: taarlab-e2cvo
test: /content/datasets/UtensilGrasp-3/test/images
train: /content/datasets/UtensilGrasp-3/train/images
val: /content/datasets/UtensilGrasp-3/valid/images


In [10]:
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

## detection

In [17]:
!yolo task=detect mode=train model=yolov8x.pt data={dataset.location}/data.yaml freeze = 10 epochs=25 imgsz=640 plots=True

Ultralytics YOLOv8.0.166 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/datasets/UtensilGrasp-3/data.yaml, epochs=25, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, kera

In [19]:
!yolo task=detect mode=val model=/content/runs/detect/train3/weights/best.pt data={dataset.location}/data.yaml imgsz=640

Ultralytics YOLOv8.0.166 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68134161 parameters, 0 gradients
val: Scanning /content/datasets/UtensilGrasp-3/valid/labels.cache... 68 images, 0 backgrounds, 0 corrupt: 100% 68/68 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 5/5 [00:07<00:00,  1.43s/it]
                   all         68         90      0.958      0.987      0.979      0.958
                bottle         68          5          1          1      0.995      0.995
          layed bottle         68          7      0.974          1      0.995      0.963
             metal box         68          8      0.985          1      0.995      0.995
            metal fork         68          4      0.775          1      0.945      0.917
           metal spoon         68          6          1      0.969      0.995      0.995
                 pfork         68         14      0.

In [20]:
!yolo task=detect mode=predict model=/content/runs/detect/train3/weights/best.pt conf= 0.1 source=/content/datasets/UtensilGrasp-3/test/images imgsz=640

Ultralytics YOLOv8.0.166 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68134161 parameters, 0 gradients

image 1/69 /content/datasets/UtensilGrasp-3/test/images/11_jpg.rf.2d6d63813ed3f0a2d3e218ed82316b76.jpg: 640x640 3 pforks, 3 pknifes, 3 pspoons, 81.6ms
image 2/69 /content/datasets/UtensilGrasp-3/test/images/13_jpg.rf.805b04bbca957eea6cb4c56177e0e903.jpg: 640x640 3 pforks, 3 pknifes, 3 pspoons, 1 square plate, 58.3ms
image 3/69 /content/datasets/UtensilGrasp-3/test/images/17_jpg.rf.1d2e7b39ed7b23054b8948d46a44f8b5.jpg: 640x640 3 pforks, 3 pknifes, 3 pspoons, 1 square plate, 58.3ms
image 4/69 /content/datasets/UtensilGrasp-3/test/images/4_jpg.rf.8b00b5b3a2ac906c341605064e64993b.jpg: 640x640 3 pforks, 2 pknifes, 4 pspoons, 1 square plate, 53.8ms
image 5/69 /content/datasets/UtensilGrasp-3/test/images/IMG_6731_JPG.rf.d29a61608a2d892bcde889598c135715.jpg: 640x640 1 pknife, 53.9ms
image 6/69 /content/datasets/UtensilGrasp-3/test/images/I

## segmentation

In [25]:
!yolo task=segment mode=train model=yolov8x-seg.pt  data={dataset.location}/data.yaml freeze = 12 epochs=30 imgsz=640

100% 137M/137M [00:19<00:00, 7.54MB/s]
Ultralytics YOLOv8.0.166 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=/content/datasets/UtensilGrasp-3/data.yaml, epochs=30, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=12, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

In [26]:
!yolo task=segment mode=val model=/content/runs/segment/train2/weights/best.pt data={dataset.location}/data.yaml

Ultralytics YOLOv8.0.166 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-seg summary (fused): 295 layers, 71731249 parameters, 0 gradients
val: Scanning /content/datasets/UtensilGrasp-3/valid/labels.cache... 68 images, 0 backgrounds, 0 corrupt: 100% 68/68 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% 5/5 [00:09<00:00,  1.98s/it]
                   all         68         90      0.939      0.976      0.984       0.96      0.939      0.976      0.984      0.917
                bottle         68          5      0.832          1      0.995      0.995      0.832          1      0.995      0.995
          layed bottle         68          7       0.97          1      0.995      0.995       0.97          1      0.995      0.995
             metal box         68          8      0.973          1      0.995      0.995      0.973          1      0.995      0.995
    

In [27]:
%cd {HOME}
!yolo task=segment mode=predict model=/content/runs/segment/train2/weights/best.pt conf=0.25 source=/content/datasets/UtensilGrasp-3/test/images save=true

/content
Ultralytics YOLOv8.0.166 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-seg summary (fused): 295 layers, 71731249 parameters, 0 gradients

image 1/69 /content/datasets/UtensilGrasp-3/test/images/11_jpg.rf.2d6d63813ed3f0a2d3e218ed82316b76.jpg: 640x640 3 pforks, 3 pknifes, 3 pspoons, 93.6ms
image 2/69 /content/datasets/UtensilGrasp-3/test/images/13_jpg.rf.805b04bbca957eea6cb4c56177e0e903.jpg: 640x640 3 pforks, 3 pknifes, 3 pspoons, 1 square plate, 76.0ms
image 3/69 /content/datasets/UtensilGrasp-3/test/images/17_jpg.rf.1d2e7b39ed7b23054b8948d46a44f8b5.jpg: 640x640 3 pforks, 3 pknifes, 3 pspoons, 1 square plate, 75.6ms
image 4/69 /content/datasets/UtensilGrasp-3/test/images/4_jpg.rf.8b00b5b3a2ac906c341605064e64993b.jpg: 640x640 3 pforks, 2 pknifes, 3 pspoons, 1 square plate, 73.1ms
image 5/69 /content/datasets/UtensilGrasp-3/test/images/IMG_6731_JPG.rf.d29a61608a2d892bcde889598c135715.jpg: 640x640 1 pknife, 71.3ms
image 6/69 /content/datasets/UtensilGrasp-

### example

In [28]:
%cd {HOME}
!yolo task=segment mode=predict model=/content/runs/segment/train2/weights/best.pt conf=0.25 source='/content/datasets/UtensilGrasp-3/test/images/4_jpg.rf.8b00b5b3a2ac906c341605064e64993b.jpg' save=true

/content
Ultralytics YOLOv8.0.166 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-seg summary (fused): 295 layers, 71731249 parameters, 0 gradients

image 1/1 /content/datasets/UtensilGrasp-3/test/images/4_jpg.rf.8b00b5b3a2ac906c341605064e64993b.jpg: 640x640 3 pforks, 2 pknifes, 3 pspoons, 1 square plate, 106.0ms
Speed: 1.9ms preprocess, 106.0ms inference, 80.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/segment/predict3
